# Latency Lab Analysis

This notebook reads `results/index.csv` and shows basic comparisons.


In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve() / "scripts"))

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from results_lib import load_index, load_samples

sns.set_theme(style="whitegrid")


In [ ]:
index = load_index()
index.head()


In [ ]:
# Pick a case to compare pinned vs unpinned (adjust as needed)
case_name = index["case"].iloc[0] if len(index) else "noop"
subset = index[index["case"] == case_name]

display(subset[["case","tags","pin_cpu","p50","p95","p99","p999","mean"]])


In [ ]:
# Long-form view of quantiles for the chosen case
if len(subset):
    plot_df = subset.melt(
        id_vars=["tags","pin_cpu"],
        value_vars=["p50","p95","p99","p999"],
        var_name="quantile",
        value_name="ns",
    )
    plt.figure(figsize=(8,4))
    sns.barplot(data=plot_df, x="quantile", y="ns", hue="pin_cpu")
    plt.title(f"{case_name} quantiles by pin_cpu")
    plt.show()


In [ ]:
# ECDF of raw samples for the first two runs (if raw data is present)
if len(index) >= 2:
    run_a = index.iloc[0]["run_dir"]
    run_b = index.iloc[1]["run_dir"]
    samples_a = load_samples(run_a)
    samples_b = load_samples(run_b)

    plt.figure(figsize=(8,4))
    sns.ecdfplot(samples_a, label=str(run_a))
    sns.ecdfplot(samples_b, label=str(run_b))
    plt.xlabel("ns")
    plt.ylabel("ECDF")
    plt.legend()
    plt.title("ECDF comparison")
    plt.show()
